# Topic Modeling
In this exercise, we will do topic modeling with gensim. Use the [topics and transformations tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html) as a reference.

In [2]:
import os
from collections import defaultdict

import gensim
import nltk

For tokenizing words and stopword removal, download the NLTK punkt tokenizer and stopwords list.

In [3]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /home/lggram/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/lggram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

First, we load the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included with gensim that contains 300 news articles of the Australian Broadcasting Corporation.

In [4]:
from gensim.test.utils import datapath
train_file = datapath('lee_background.cor')
articles_orig = open(train_file).read().splitlines()

Preprocess the text by lowercasing, removing stopwords, stemming, and removing rare words.

In [19]:
# define stopword list
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords = stopwords | {'\"', '\'', '\'\'', '`', '``', '\'s'}

# initialize stemmer
stemmer = nltk.stem.PorterStemmer()

def preprocess(article):
    # tokenize
    article = nltk.word_tokenize(article)

    # lowercase all words
    article = list(map(str.lower, article))

    # remove stopwords
    article = [word for word in article if word not in stopwords]

    # optional: stem
    article = [stemmer.stem(word) for word in article]
    return article

articles = [preprocess(article) for article in articles_orig]

# create the dictionary and corpus objects that gensim uses for topic modeling
dictionary = gensim.corpora.Dictionary(articles)

# remove words that occur in less than 2 documents, or more than 50% of documents
dictionary.filter_extremes(no_below=2, no_above=0.5)
temp = dictionary[0]  # load the dictionary by calling it once
corpus_bow = [dictionary.doc2bow(article) for article in articles]
print(corpus_bow)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 8), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 2), (53, 3), (54, 3), (55, 1), (56, 2), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 8), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 3), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 5), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 3), (102, 1), (103, 1), (104, 1), (105, 3), (106, 1), (107, 1), (108, 1), (109, 1), (110, 4)


Now we create a TF-IDF model and transform the corpus into TF-IDF vectors.

In [30]:
tfidf = gensim.models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]
corpus_tfidf[0]

[(0, 0.045055071138122856),
 (1, 0.048886979465828866),
 (2, 0.09375399876622974),
 (3, 0.06781504448328635),
 (4, 0.08078452162475803),
 (5, 0.10134065654795091),
 (6, 0.058730707569876556),
 (7, 0.07269531841603805),
 (8, 0.03468336276559477),
 (9, 0.07660928255691886),
 (10, 0.06936672553118954),
 (11, 0.06363980541544716),
 (12, 0.14639572768607376),
 (13, 0.06561120606131572),
 (14, 0.05734407327660557),
 (15, 0.21959359152911065),
 (16, 0.08616734098450858),
 (17, 0.03352561206466457),
 (18, 0.05163007712346614),
 (19, 0.08616734098450858),
 (20, 0.04649508920613628),
 (21, 0.08078452162475803),
 (22, 0.058730707569876556),
 (23, 0.044374596135133865),
 (24, 0.048886979465828866),
 (25, 0.05371122002149018),
 (26, 0.047258909560093466),
 (27, 0.061856456607300554),
 (28, 0.05163007712346614),
 (29, 0.08616734098450858),
 (30, 0.056053147633726),
 (31, 0.061856456607300554),
 (32, 0.14639572768607376),
 (33, 0.047258909560093466),
 (34, 0.06022838670156517),
 (35, 0.06363980541544

Now we train an [LDA model](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html) with 10 topics on the TF-IDF corpus. Save it to a variable `model_lda`.

In [57]:
model_lda = gensim.models.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=10)

Let's inspect the first 5 topics of our model.

In [58]:
top_topics = model_lda.top_topics(corpus_tfidf)
corpus_lda = model_lda[corpus_tfidf] 
model_lda.print_topics(5)

[(6,
  '0.002*"attack" + 0.002*"us" + 0.002*"arrest" + 0.002*"farmer" + 0.002*"radio" + 0.002*"white" + 0.002*"rate" + 0.002*"cut" + 0.002*"tape" + 0.002*"boat"'),
 (8,
  '0.002*"hospit" + 0.002*"health" + 0.002*"adelaid" + 0.002*"car" + 0.002*"man" + 0.002*"children" + 0.001*"differ" + 0.001*"lockett" + 0.001*"peopl" + 0.001*"test"'),
 (1,
  '0.002*"palestinian" + 0.002*"mr" + 0.002*"fire" + 0.002*"law" + 0.002*"forc" + 0.002*"arafat" + 0.002*"rate" + 0.002*"report" + 0.002*"new" + 0.002*"area"'),
 (9,
  '0.003*"isra" + 0.002*"palestinian" + 0.002*"south" + 0.002*"road" + 0.002*"fire" + 0.002*"per" + 0.002*"test" + 0.002*"australian" + 0.002*"lee" + 0.002*"cent"'),
 (3,
  '0.003*"senat" + 0.002*"taliban" + 0.002*"presid" + 0.002*"product" + 0.002*"club" + 0.002*"count" + 0.002*"compani" + 0.002*"qanta" + 0.002*"saa" + 0.002*"asic"')]

We see the 5 topics with the highest importance. For each topic, the 10 most important words are shown, together with their coefficient of "alignment" to the topic.

## Document Similarity
We now use our LDA model to compare the similarity of new documents (*queries*) to documents in our collection.

First, create an index of the news articles in our corpus. Use the `MatrixSimilarity` transformation as described in gensim's [similarity queries tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html).

In [59]:
index = gensim.similarities.MatrixSimilarity(corpus_lda)

Now, write a function that takes a query string as input and returns the LDA representation for it. Make sure to apply the same preprocessing as we did to the documents.

In [54]:
def query_lda(query):
    query_tokens = preprocess(query)
    query_bow = dictionary.doc2bow(query_tokens)
    query_tfidf = tfidf[query_bow]
    return model_lda[query_tfidf]

Print the top 5 most similar documents, together with their similarities, using your index created above.

In [60]:
vec = query_lda("Qantas and Virgin size?")
sims = index[vec]
sims = sorted(enumerate(sims), key=lambda item: -item[1])[:5]
for doc_position, doc_score in sims:
    print(doc_score, articles_orig[doc_position])

0.9948478 In the United States, Australian actress Nicole Kidman has been nominated for two Golden Globe best actor awards for her roles in the Australian-made musical "Moulin Rouge", and in her new thriller "The Others". "Moulin Rouge" also is one of two pictures leading the Golden Globe nominations, with six possible awards. It is vying for best musical or comedy picture of 2002, best actress in a comedy or musical, best actor in the same category for Ewen McGregor, best director for Baz Luhrmann, best original score and best original song. The other film to pick up six nominations is the Ron Howard directed "A Beautiful Mind" starring Oscar winner Russell Crowe. Crowe was nominated as best actor in a drama for his portrayal of a troubled math genius in the film. 
0.9948342 Defence Minister Robert Hill has confirmed Australian troops arrived in Afghanistan this morning. Senator Hill says it is an advance party and the rest of the troops will arrive within the next few days. He says A

Run your code again, now training an LDA model with 100 topics. Do you see a qualitative difference in the top-5 most similar documents?

Better quality with 100 topics